In [ ]:
import os
import sys
import importlib
from pathlib import Path

mod = sys.modules[__name__]

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

for _ in range(5):

    parent_path = str(Path.cwd().parents[_]).split('/')[-1]

    if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

        code_dir = str(Path.cwd().parents[_])

        if code_dir is not None:
            break

# %load_ext autoreload
# %autoreload 2

In [ ]:
# MAIN DIR
main_dir = f'{str(Path(code_dir).parents[0])}/'

# code_dir
code_dir = f'{code_dir}/'
sys.path.append(code_dir)

# scraping dir
scraped_data = f'{code_dir}scraped_data/'

# data dir
data_dir = f'{code_dir}data/'

# lang models dir
llm_path = f'{data_dir}Language Models'

# sites
site_list=['Indeed', 'Glassdoor', 'LinkedIn']

# columns
cols=['Sector', 
      'Sector Code', 
      'Gender', 
      'Age', 
      'Language', 
      'Dutch Requirement', 
      'English Requirement', 
      'Gender_Female', 
      'Gender_Mixed', 
      'Gender_Male', 
      'Age_Older', 
      'Age_Mixed', 
      'Age_Younger', 
      'Gender_Num', 
      'Age_Num', 
      '% Female', 
      '% Male', 
      '% Older', 
      '% Younger']

int_variable: str = 'Job ID'
str_variable: str = 'Job Description'
gender: str = 'Gender'
age: str = 'Age'
language: str = 'en'
str_cols = ['Search Keyword', 'Platform', 'Job ID', 'Job Title', 'Company Name', 'Location', 'Job Description', 'Company URL', 'Job URL', 'Tracking ID']
nan_list = [None, 'None', '', ' ', [], -1, '-1', 0, '0', 'nan', np.nan, 'Nan']


In [ ]:
import os
import sys
import re
import time
import json
import csv
import glob
import pickle
import pandas as pd
from pathlib import Path
from googletrans import Translator
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from spacy.pipeline import Sentencizer


In [ ]:
# This is a manually collected dictionary of incorrect/faulty keywords in scraped site data
with open(f'{scraped_data}CBS/Data/keyword_trans_dict.txt') as f:
    keyword_trans_dict = json.load(f)


In [ ]:
# 111 words to fix
len(keyword_trans_dict)


In [ ]:
def fix_broken_linkedin_files(glob_path):
    fix_list = []
    data_dict = {}
    data_list = []

    if glob_path.endswith('.json'):

        with open(glob_path, encoding = 'utf-8') as csv_file_handler:
            csv_reader = csv.DictReader(csv_file_handler)

            for rows in csv_reader:
                first_key = str(list(rows.keys())[0])
                key = rows[first_key]
                data_dict[key] = rows

        for num in data_dict:
            data_list.append(data_dict[num])

        with open(glob_path, 'w', encoding = 'utf-8') as json_file_handler:
            json_file_handler.write(json.dumps(data_list, indent = 4))
    
    return data_list


In [ ]:
def fix_keywords(df_temp):
    if len(df_temp) > 0 and isinstance(df_temp, pd.DataFrame):
        for key, value in keyword_trans_dict.items():
            df_temp.loc[
                df_temp['Search Keyword'].astype(str).apply(
                lambda x: x.lower().strip()
                ) == str(key).lower().strip(), 'Search Keyword'
            ] = str(value).lower().strip()

        unfixed = df_temp.loc[
            df_temp['Search Keyword'].astype(str).apply(lambda x: x.lower().strip()).isin([x.lower().strip() for x in list(keyword_trans_dict.keys())])
        ]

        if len(unfixed) != 0:
            for key, value in keyword_trans_dict.items():
                for idx, row in df_temp.iterrows():
                    if row['Search Keyword'].astype(str).lower().strip() == str(key).lower().strip():
                        df_temp.loc[idx, 'Search Keyword'] = str(value).lower().strip()
    

    return df_temp


#### Read paths

In [ ]:
glob_paths = []

for site in site_list:
    glob_paths.extend(glob.glob(f'{scraped_data}/{site}/Data/*.json')+glob.glob(f'{scraped_data}/{site}/Data/*.csv'))


In [ ]:
# 955 json and csv files
len(glob_paths)


#### Use paths to open files, fix keywords, and drop unneeded columns

In [ ]:
# Fix list catches all incorrect/faculty keyword search terms

fix_list = []

# Appended data catches all the fixed and cleaned dfs
appended_data = []

for glob_path in glob_paths:

    if glob_path.endswith('.json'):
        try:
            df_temp = pd.read_json(glob_path).reset_index(drop=True)
        except ValueError:
            fix_list.append(glob_path)
            if 'scraped_data/LinkedIn/Data/linkedin_jobs_df_' in glob_path:
                data_json = fix_broken_linkedin_files(glob_path)
                try:
                    df_temp = pd.read_json(glob_path).reset_index(drop=True)
                except ValueError:
                    fix_list.append(glob_path)
    elif glob_path.endswith('.csv'):
        df_temp = pd.read_csv(glob_path).reset_index(drop=True)

    if len(df_temp) > 0 and isinstance(df_temp, pd.DataFrame):
        df_temp = fix_keywords(df_temp)
        df_temp.reset_index(drop=True, inplace=True)
        df_temp.drop(columns=cols, axis='columns', inplace=True, errors='ignore')
        df_temp.drop(
        df_temp.columns[
                df_temp.columns.str.contains(
                    'unnamed|index|level', regex=True, case=False, flags=re.I
                )
            ],
            axis='columns',
            inplace=True,
            errors='ignore',
        )
#         df_temp.drop(columns=df_temp.filter(regex=re.compile(r"^unnamed", re.IGNORECASE)).columns, axis='columns', inplace=True, errors='ignore')
    
        if glob_path.endswith('.json'):
            df_temp.to_json(glob_path, orient='records')
        elif glob_path.endswith('.csv'):
            df_temp.to_csv(glob_path, index=False)

        appended_data.append(df_temp.reset_index(drop=True))

# Concatonate list of dfs into one large df_jobs
df_jobs = pd.concat(appended_data).reset_index(drop=True)

# Save df_jobs to file
if len(df_jobs) > 0 and isinstance(df_jobs, pd.DataFrame):
    df_jobs.to_pickle(f'{data_dir}df_jobs_raw.pkl')
    
    df_jobs.to_csv(f'{data_dir}df_jobs_raw.csv', index=False)
else:
    print(f'ERORR: LENGTH OF DF = {len(df_jobs)}')


In [ ]:
# List of dfs, len = 527
len(appended_data)


In [ ]:
# len = 204394
len(df_jobs)


In [ ]:
# If we couldn't fix some keywords, we add them to list fix_list and write to file
if len(fix_list) != 0:
    print('Some keywords to fix!')
    with open(f'{data_dir}fix_list.txt', 'w') as f:
        json.dump(fix_list, f)


### START HERE IF SOURCING FROM DF_RAW
### PLEASE SET CORRECT DIRECTORY PATHS BELOW

In [ ]:
import os
import sys
import importlib
from pathlib import Path

mod = sys.modules[__name__]

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

for _ in range(5):

    parent_path = str(Path.cwd().parents[_]).split('/')[-1]

    if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

        code_dir = str(Path.cwd().parents[_])

        if code_dir is not None:
            break

# %load_ext autoreload
# %autoreload 2

In [ ]:
# MAIN DIR
main_dir = f'{str(Path(code_dir).parents[0])}/'

# code_dir
code_dir = f'{code_dir}/'
sys.path.append(code_dir)

# scraping dir
scraped_data = f'{code_dir}scraped_data/'

# data dir
data_dir = f'{code_dir}data/'

# lang models dir
llm_path = f'{data_dir}Language Models'

# sites
site_list=['Indeed', 'Glassdoor', 'LinkedIn']

# columns
cols=['Sector', 
      'Sector Code', 
      'Gender', 
      'Age', 
      'Language', 
      'Dutch Requirement', 
      'English Requirement', 
      'Gender_Female', 
      'Gender_Mixed', 
      'Gender_Male', 
      'Age_Older', 
      'Age_Mixed', 
      'Age_Younger', 
      'Gender_Num', 
      'Age_Num', 
      '% Female', 
      '% Male', 
      '% Older', 
      '% Younger']

int_variable: str = 'Job ID'
str_variable: str = 'Job Description'
gender: str = 'Gender'
age: str = 'Age'
language: str = 'en'
str_cols = ['Search Keyword', 'Platform', 'Job ID', 'Job Title', 'Company Name', 'Location', 'Job Description', 'Company URL', 'Job URL', 'Tracking ID']
nan_list = [None, 'None', '', ' ', [], -1, '-1', 0, '0', 'nan', np.nan, 'Nan']


In [ ]:
import os
import sys
import re
import time
import json
import csv
import glob
import pickle
import pandas as pd
import googletrans
from pathlib import Path
from googletrans import Translator
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from spacy.pipeline import Sentencizer


In [ ]:
df_jobs = pd.read_pickle(f'{data_dir}df_jobs_raw.pkl').reset_index(drop=True)


In [ ]:
# len = 204394
len(df_jobs)


In [ ]:
df_jobs.info()


In [ ]:
df_jobs.columns

In [ ]:
# Clean columns
df_jobs.columns = df_jobs.columns.to_series().apply(lambda x: str(x).strip())


In [ ]:
# Drop NA
df_jobs.dropna(axis='index', how='all', inplace=True)
df_jobs.dropna(axis='columns', how='all', inplace=True)


In [ ]:
# len = 204394
len(df_jobs)


In [ ]:
# Drop duplicates on subset of 'Job Description'
df_jobs.drop_duplicates(subset=['Company Name', 'Job Description'], keep='first', ignore_index=True, inplace=True)


In [ ]:
# len = 64694
len(df_jobs)


In [ ]:
# Remove any rows with missing 'Job Description'
df_jobs.drop(
    df_jobs.index[
        df_jobs['Job Description'].isin(nan_list)
    ], 
    axis='index',
    inplace=True,
    errors='ignore'

)

# Save df_jobs to file
if len(df_jobs) > 0 and isinstance(df_jobs, pd.DataFrame):
    df_jobs.to_pickle(f'{data_dir}df_jobs_raw_dropped.pkl')
    
    df_jobs.to_csv(f'{data_dir}df_jobs_raw_dropped.csv', index=False)
else:
    print(f'ERORR: LENGTH OF DF = {len(df_jobs)}')


### START HERE IF SOURCING FROM DF_RAW_DROPPED
### PLEASE SET CORRECT DIRECTORY PATHS BELOW


In [1]:
import os
import sys
import importlib
from pathlib import Path

mod = sys.modules[__name__]

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

for _ in range(5):

    parent_path = str(Path.cwd().parents[_]).split('/')[-1]

    if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

        code_dir = str(Path.cwd().parents[_])

        if code_dir is not None:
            break

# %load_ext autoreload
# %autoreload 2

In [2]:
# MAIN DIR
main_dir = f'{str(Path(code_dir).parents[0])}/'

# code_dir
code_dir = f'{code_dir}/'
sys.path.append(code_dir)

# scraping dir
scraped_data = f'{code_dir}scraped_data/'

# data dir
data_dir = f'{code_dir}data/'

# lang models dir
llm_path = f'{data_dir}Language Models'

# sites
site_list=['Indeed', 'Glassdoor', 'LinkedIn']

# columns
cols=['Sector', 
      'Sector Code', 
      'Gender', 
      'Age', 
      'Language', 
      'Dutch Requirement', 
      'English Requirement', 
      'Gender_Female', 
      'Gender_Mixed', 
      'Gender_Male', 
      'Age_Older', 
      'Age_Mixed', 
      'Age_Younger', 
      'Gender_Num', 
      'Age_Num', 
      '% Female', 
      '% Male', 
      '% Older', 
      '% Younger']

int_variable: str = 'Job ID'
str_variable: str = 'Job Description'
gender: str = 'Gender'
age: str = 'Age'
language: str = 'en'
str_cols = ['Search Keyword', 'Platform', 'Job ID', 'Job Title', 'Company Name', 'Location', 'Job Description', 'Company URL', 'Job URL', 'Tracking ID']
nan_list = [None, 'None', '', ' ', [], -1, '-1', 0, '0', 'nan', np.nan, 'Nan']


In [3]:
import os
import sys
import re
import time
import json
import csv
import glob
import pickle
import pandas as pd
import googletrans
from pathlib import Path
from googletrans import Translator
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from spacy.pipeline import Sentencizer


In [4]:
df_jobs = pd.read_pickle(f'{data_dir}df_jobs_raw_dropped.pkl').reset_index(drop=True)


In [5]:
# 64684
len(df_jobs)


64684

In [6]:
df_jobs.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64684 entries, 0 to 64683
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Search Keyword     64684 non-null  object 
 1   Platform           64684 non-null  object 
 2   Job ID             64684 non-null  object 
 3   Job Title          64684 non-null  object 
 4   Company Name       64679 non-null  object 
 5   Location           64684 non-null  object 
 6   Job Description    64684 non-null  object 
 7   Rating             4130 non-null   float64
 8   Employment Type    63959 non-null  object 
 9   Company URL        61360 non-null  object 
 10  Job URL            64684 non-null  object 
 11  Job Age            64684 non-null  object 
 12  Job Age Number     64684 non-null  object 
 13  Collection Date    64684 non-null  object 
 14  Data Row           60551 non-null  float64
 15  Tracking ID        60551 non-null  object 
 16  Industry           612

In [14]:
df_jobs = pd.read_pickle(f'{data_dir}df_jobs_raw_language_detected.pkl').reset_index(drop=True)


In [15]:
# 64684
len(df_jobs)


64684

In [16]:
df_jobs['Language'].isna().sum()

62706

In [17]:
df_jobs['Language'].value_counts()

en    1785
nl     193
Name: Language, dtype: int64

In [ ]:
translator = Translator()
googletrans_readtime_error = googletrans.client.httpx._client.httpcore._exceptions.ReadTimeout

# try:
#     time.sleep(60)
#     df_jobs['Language'] = df_jobs['Job Description'].apply(lambda x: translator.detect(str(x).lower().strip()).lang)
# except:
#     time.sleep(3600)
#     df_jobs['Language'] = df_jobs['Job Description'].apply(lambda x: translator.detect(str(x).lower().strip()).lang)

for idx, row in df_jobs.iterrows():
    if len(str(row['Job Description'])) != 0 or not isinstance(row['Language'], str) or row['Language'] in nan_list:

        try:
            print(f'Row {idx}: Translation in progress.')
            time.sleep(60)
            df_jobs.loc[idx, 'Language'] = str(translator.detect(str(row['Job Description']).lower().strip()).lang)
        except googletrans_readtime_error:
            print(f'Row {idx}: Sleeping for an hour.')
            time.sleep(3600)
            print(f'Row {idx}: Done sleeping.')
            df_jobs.loc[idx, 'Language'] = str(translator.detect(str(row['Job Description']).lower().strip()).lang)

if len(df_jobs) > 0 and isinstance(df_jobs, pd.DataFrame):
    df_jobs.to_pickle(f'{data_dir}df_jobs_raw_language_detected.pkl')
    
    df_jobs.to_csv(f'{data_dir}df_jobs_raw_language_detected.csv', index=False)
else:
    print(f'ERORR: LENGTH OF DF = {len(df_jobs)}')


Row 0: Translation in progress.
Row 1: Translation in progress.
Row 2: Translation in progress.
Row 3: Translation in progress.
Row 4: Translation in progress.
Row 5: Translation in progress.
Row 6: Translation in progress.
Row 7: Translation in progress.
Row 8: Translation in progress.
Row 9: Translation in progress.
Row 10: Translation in progress.
Row 11: Translation in progress.
Row 12: Translation in progress.
Row 13: Translation in progress.
Row 14: Translation in progress.
Row 15: Translation in progress.
Row 16: Translation in progress.
Row 17: Translation in progress.
Row 18: Translation in progress.
Row 19: Translation in progress.
Row 20: Translation in progress.
Row 21: Translation in progress.
Row 22: Translation in progress.
Row 23: Translation in progress.
Row 24: Translation in progress.
Row 25: Translation in progress.
Row 26: Translation in progress.
Row 27: Translation in progress.
Row 28: Translation in progress.
Row 29: Translation in progress.
Row 30: Translation 

In [ ]:
# Drop non-english job descriptions
# df_jobs.drop(df_jobs.index[df_jobs['Language'] == str(language)], axis='index', inplace=True, errors='ignore')


In [ ]:
if len(df_jobs) > 0 and isinstance(df_jobs, pd.DataFrame):
    df_jobs.to_pickle(f'{data_dir}df_jobs_raw_language_detected.pkl')
    
    df_jobs.to_csv(f'{data_dir}df_jobs_raw_language_detected.csv', index=False)
else:
    print(f'ERORR: LENGTH OF DF = {len(df_jobs)}')

In [ ]:
# df_jobs = pd.read_pickle(f'{data_dir}df_jobs_raw_language_detected.pkl').reset_index(drop=True)


In [ ]:
# Find and count unique search keywords
search_keywords = list(set(df_jobs['Search Keyword'].to_list()))


In [ ]:
len(search_keywords)


In [ ]:
search_keywords


In [ ]:
job_descriptions = list(set(df_jobs['Job Description'].to_list()))

In [ ]:
len(job_descriptions)

In [ ]:
# job_descriptions

In [ ]:
# Load NLK
nltk_path = f'{llm_path}/nltk'
nltk.data.path.append(nltk_path)

nltk.download('words', download_dir = nltk_path)
nltk.download('punkt', download_dir = nltk_path)
nltk.download('stopwords', download_dir = nltk_path)


In [ ]:
# Load SpaCY
sentencizer = Sentencizer(punct_chars=[',,', ',,,', ',,,,'])


In [ ]:
for job_description in job_descriptions:
    print([doc for doc in sentencizer()])


In [ ]:
job_sentences = []
for job_description in job_descriptions:
    job_sentences.extend(sent_tokenize(job_description))


In [ ]:
job_sentences[0].split('\n')



In [ ]:
# Use spacy with nlp to sent tokenize

In [ ]:
nlp = spacy.load('en_core_web_sm')